<a href="https://colab.research.google.com/github/ivoryRabbit/RecSys/blob/master/2_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K Nearest Neighbor

## Experiment

In [18]:
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from typing import Callable, Tuple, List

from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

from tensorflow.keras.utils import get_file
import zipfile

In [19]:
def load_data(data_size : str) -> pd.DataFrame:
    ''' load Movie Lens data '''

    if data_size == '1m':
        fname = 'ml-1m.zip'
        data = 'ml-1m/ratings.dat'
    elif data_size == '10m':
        fname = 'ml-10m.zip'
        data = 'ml-10M100K/ratings.dat'
    elif data_size == '20m':
        fname = 'ml-20m.zip'
        data = 'ml-20m/ratings.csv'
    elif data_size == '25m':
        fname = 'ml-25m.zip'
        data = 'ml-25m/ratings.csv'
    if not glob.glob(data):
        origin = f'http://files.grouplens.org/datasets/movielens/{fname}'
        file = get_file(fname, origin)
        zip_ref = zipfile.ZipFile(file, 'r')
        zip_ref.extractall()

    col_names = ['userId', 'movieId', 'rating', 'timestamp']
    if data_size in ['20m', '25m']:
        ratings = pd.read_csv(data, engine = 'python')
    else:
        ratings = pd.read_csv(data, sep = '|', delimiter = '::', names = col_names, engine = 'python')
    print(ratings.shape)
    return ratings

In [20]:
ratings = load_data('1m')
ratings.head()

(1000209, 4)


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [21]:
n_user = ratings.userId.nunique()
print(f'# of user = {n_user}')

n_item = ratings.movieId.nunique()
print(f'# of item = {n_item}')

# of user = 6040
# of item = 3706


In [22]:
def train_valid_test_split(df: pd.DataFrame) -> pd.DataFrame:
    train_user, test_user = train_test_split(df.userId.unique(), test_size = 0.2, random_state = 777)
    valid_user, test_user = train_test_split(test_user, test_size = 0.5, random_state = 777)

    train, valid, test = map(lambda x: df[df.userId.isin(x)], (train_user, valid_user, test_user))
    train, valid, test = map(lambda df: df.reset_index(drop = True), (train, valid, test))
    return train, valid, test

def query_answer_split(df: pd.DataFrame) -> pd.DataFrame:
    timeorder = df.groupby(by = 'userId')['timestamp'].rank(method = 'first', ascending = True)
    seen_cnts = df.groupby(by = 'userId', as_index = False).agg(seen_cnts = ('movieId', 'count'))
    df = df.merge(seen_cnts, how = 'left', on = 'userId')
    df = df.assign(timeorder = timeorder)
    df = df.assign(split_type = np.where(df.timeorder < df.seen_cnts * 0.8, 'query', 'answer'))
    query = df[df.split_type == 'query']
    answer = df[df.split_type == 'answer']
    answer = answer[answer.userId.isin(query.userId.unique())]
    query, answer = map(lambda df: df.drop(columns = ['timeorder', 'seen_cnts', 'split_type']), (query, answer))
    query, answer = map(lambda df: df.reset_index(drop = True), (query, answer))
    return query, answer

def cut_off(df: pd.DataFrame, threshold = 4) -> pd.DataFrame:
    return df[df.rating >= threshold].reset_index(drop = True)

def make_warm(df: pd.DataFrame, threshold = 5) -> pd.DataFrame: # remove cold starters
    positive = df.groupby('userId')['movieId'].nunique()
    positive = positive.index[positive >= threshold]
    return df[df.userId.isin(positive)].reset_index(drop = True)

def list_aggregation(df: pd.DataFrame) -> pd.DataFrame:
    return df.groupby('userId', as_index = False)[['movieId', 'rating']].agg(list)

In [23]:
class KNN:
    def __init__(self, n_user: int, n_item: int):
        self.n_user = n_user
        self.n_item = n_item

    def id2idx(self, df: pd.DataFrame) -> pd.DataFrame:
        return df.assign(userId = lambda x: x.userId.map(self.user_id2idx), 
                         movieId = lambda x: x.movieId.map(self.item_id2idx))

    def idx2id(self, df: pd.DataFrame) -> pd.DataFrame:
        return df.assign(userId = lambda x: x.userId.apply(lambda x: self.user_idx2id[x]), 
                         movieId = lambda x: x.movieId.apply(lambda x: self.item_idx2id[x]))
    
    def fit(self, df: pd.DataFrame):
        self.user_idx2id = df.userId.unique() # list[index] = id
        self.user_id2idx = {e: i for i, e in enumerate(self.user_idx2id)} # dict[id] = index
        self.item_idx2id = df.movieId.unique() # list[index] = id
        self.item_id2idx = {e: i for i, e in enumerate(self.item_idx2id)} # dict[id] = index
        _train = self.id2idx(df)

        row = _train.userId
        col = _train.movieId
        data = np.ones(shape = _train.index.size)
        shape = (len(self.user_idx2id), self.n_item)
        
        self.mat = csr_matrix((data, (row, col)), shape = shape)
        self.mat_T = self.mat.T
    
    def predict(self, df:pd.DataFrame, N: int, similarity = 'dot') -> List:
        pred = []
        profile = df.movieId.apply(lambda x: [self.item_id2idx.get(y) for y in x if self.item_id2idx.get(y)])
        for idx in tqdm(df.index):
            y_true = np.zeros(shape = (self.n_item, ))
            pos = profile[idx]
            y_true[pos] = 1

            sim = self.mat.dot(y_true)
            a = self.mat.dot(np.ones(shape = (self.n_item, )))
            b = np.sum(y_true)
            if similarity == 'cosine':
                sim /= np.clip(np.sqrt(a * b), a_min = 1, a_max = np.inf)
            elif similarity == 'jaccard':
                sim /= (a + b - sim)
            elif similarity == 'euclidean':
                sim = np.sqrt(a + b - 2 * sim)
            y_pred = self.mat_T.dot(sim)

            cand = np.argsort(np.where(y_true == 1., -1., y_pred))[:-N-1:-1]
            cand = self.item_idx2id[cand]
            pred.append({'userId': df.at[idx, 'userId'],
                         'movieId': cand})
        return pd.DataFrame(pred)

In [24]:
class evaluate:
    def __init__(self, true: pd.DataFrame, pred: pd.DataFrame):
        self.true = true
        self.pred = pred
        self.max_K = 10000
        self.idcg = np.cumsum([1.0 / np.log(i+2) for i in range(self.max_K)])

    def _recall(self, gt: List, rec: List, K = None) -> float:
        K = K if K else self.max_K
        res = [r for r in rec[:K] if r in gt]
        return len(res) / np.min([K, len(gt)])
    
    def _precision(self, gt: List, rec: List, K = None) -> float:
        K = K if K else self.max_K
        res = [r for r in rec[:K] if r in gt]
        return len(res) / len(rec[:K])

    def _AP(self, gt: List, rec: List, K = None) -> float: # Average Precision
        K = K if K else self.max_K
        res = 0
        for i, r in enumerate(rec[:K]):
            if r in gt:
                res += self._precision(gt, rec[:K], i+1)
        return res / np.min([K, len(gt)])

    def _RR(self, gt: List, rec: List, K = None) -> float: # Reciprocal Rank
        K = K if K else self.max_K
        for i, r in enumerate(rec[:K]):
            if r in gt:
                return  1.0 / (i+1)
        return 0

    def _nDCG(self, gt: List, rec: List, K = None) -> float: # normalized Discounted Cumulative Gain
        K = K if K else self.max_K
        dcg = 0.0
        for i, r in enumerate(rec[:K]):
            if r in gt:
                dcg += 1.0 / np.log(i+2)
        idcg = self.idcg[min([len(gt), K])-1]
        return dcg / idcg
    
    def __call__(self, K = None):
        self.K = K
        self.recall = 0.0
        self.precision = 0.0
        self.MAP = 0.0
        self.MRR = 0.0
        self.nDCG = 0.0
        n = self.true.index.size
        for gt, rec in zip(tqdm(self.true.movieId), self.pred.movieId):
            self.recall += self._recall(gt, rec, K) / n
            self.precision += self._precision(gt, rec, K) / n
            self.MAP += self._AP(gt, rec, K) / n
            self.MRR += self._RR(gt, rec, K) / n
            self.nDCG += self._nDCG(gt, rec, K) / n

    def print_all(self):
        K = '@' + str(self.K) if self.K else ''
        print(f'{"Recall":>12}{K} : {self.recall:.5f}',
              f'\n{"Precision":>12}{K} : {self.precision:.5f}',
              f'\n{"MAP":>12}{K} : {self.MAP:.5f}',
              f'\n{"nRR":>12}{K} : {self.MRR:.5f}',
              f'\n{"nDCG":>12}{K} : {self.nDCG:.5f}')

In [25]:
ratings = cut_off(ratings)

train, _, test = train_valid_test_split(ratings)
train = make_warm(train)

test_q, test_a = query_answer_split(test)
test_q, test_a = map(list_aggregation, (test_q, test_a))

### Remark

- Since A and B are binary vectors, it satisfies $ A ^2 = A $ by the element-wise product. 

- Futher, note that the dot product "$\cdot$" satisfies that $ A \cdot B = \sum AB $.

In [26]:
model = KNN(n_user, n_item)
model.fit(train)

### 1. Dot Product Similarity

$$ \text{DotSim}(A, B) := A \cdot B$$

In [27]:
pred = model.predict(test_q, N = 100)
pred.head(5)

,userId,movieId
0,5,"[1196, 260, 318, 2762, 1617, 1198, 2028, 527, ..."
1,10,"[1196, 260, 2858, 2571, 593, 2028, 318, 608, 5..."
2,15,"[1196, 260, 593, 589, 608, 1210, 296, 1270, 16..."
3,45,"[260, 1196, 2858, 1198, 2396, 593, 318, 1197, ..."
4,60,"[1196, 260, 2571, 2858, 1198, 2762, 2028, 1210..."


In [28]:
scores = evaluate(test_a, pred)

scores(K = 20)
scores.print_all()

scores(K = 100)
scores.print_all()


      Recall@20 : 0.12569 
   Precision@20 : 0.07666 
         MAP@20 : 0.04343 
         nRR@20 : 0.24152 
        nDCG@20 : 0.11315



      Recall@100 : 0.29400 
   Precision@100 : 0.04725 
         MAP@100 : 0.05232 
         nRR@100 : 0.24773 
        nDCG@100 : 0.17989


### 2 Cosine Similarity

$$ \text{CosSim}(A, B) := \cos(\angle AB) = \frac{A \cdot B}{|A||B|} = \frac{A \cdot B}{\sqrt{\sum A}\sqrt{\sum B}}$$

In [29]:
pred = model.predict(test_q, N = 100, similarity = 'cosine')
pred.head(5)

,userId,movieId
0,5,"[2762, 260, 1196, 318, 2028, 1617, 527, 1198, ..."
1,10,"[260, 1196, 2858, 2571, 2028, 593, 318, 527, 6..."
2,15,"[1196, 260, 593, 589, 1210, 608, 1617, 296, 12..."
3,45,"[2858, 260, 1196, 2396, 1198, 593, 318, 1197, ..."
4,60,"[2858, 260, 1196, 2571, 2762, 2028, 1210, 1198..."


In [30]:
scores = evaluate(test_a, pred)

scores(K = 20)
scores.print_all()

scores(K = 100)
scores.print_all()


      Recall@20 : 0.12682 
   Precision@20 : 0.07682 
         MAP@20 : 0.04528 
         nRR@20 : 0.25183 
        nDCG@20 : 0.11589



      Recall@100 : 0.30295 
   Precision@100 : 0.04793 
         MAP@100 : 0.05456 
         nRR@100 : 0.25845 
        nDCG@100 : 0.18535


### 3 Jaccard Similarity

$$ \text{JacSim}(A, B) := \frac{A \cdot B}{\sum A + \sum B + A \cdot B}$$

In [31]:
pred = model.predict(test_q, N = 100, similarity = 'jaccard')
pred.head(5)

,userId,movieId
0,5,"[2762, 1196, 260, 318, 2028, 1617, 527, 2396, ..."
1,10,"[260, 1196, 2858, 2571, 2028, 593, 318, 608, 5..."
2,15,"[1196, 260, 593, 589, 1210, 608, 1617, 296, 12..."
3,45,"[2858, 260, 1196, 2396, 1198, 593, 1197, 318, ..."
4,60,"[2858, 260, 2571, 1196, 2762, 2028, 1210, 589,..."


In [32]:
scores = evaluate(test_a, pred)

scores(K = 20)
scores.print_all()

scores(K = 100)
scores.print_all()


      Recall@20 : 0.12771 
   Precision@20 : 0.07732 
         MAP@20 : 0.04732 
         nRR@20 : 0.26149 
        nDCG@20 : 0.11884



      Recall@100 : 0.31048 
   Precision@100 : 0.04866 
         MAP@100 : 0.05705 
         nRR@100 : 0.26827 
        nDCG@100 : 0.19064


### 4 Euclidean Distance

$$
\begin{aligned}
\text{Dist}(A, B) &:= \sqrt{\sum(A - B)^2}
\\ &= \sqrt{\sum (A^2 - 2AB + B^2)}
\\ &= \sqrt{\sum (A + B - 2AB)}
\\ &= \sqrt{\sum A + \sum B - 2 A \cdot B}
\end{aligned}
$$

In [33]:
pred = model.predict(test_q, N = 100, similarity = 'euclidean')
pred.head(5)

,userId,movieId
0,5,"[260, 1196, 1198, 2028, 2762, 318, 1210, 527, ..."
1,10,"[2858, 260, 1196, 593, 2028, 2571, 608, 318, 5..."
2,15,"[260, 1196, 593, 608, 1210, 589, 858, 1270, 11..."
3,45,"[2858, 260, 1196, 1198, 593, 2028, 2571, 2762,..."
4,60,"[2858, 260, 1196, 1198, 593, 2028, 2571, 608, ..."


In [34]:
scores = evaluate(test_a, pred)

scores(K = 20)
scores.print_all()

scores(K = 100)
scores.print_all()


      Recall@20 : 0.10845 
   Precision@20 : 0.06813 
         MAP@20 : 0.03413 
         nRR@20 : 0.19874 
        nDCG@20 : 0.09490



      Recall@100 : 0.26855 
   Precision@100 : 0.04364 
         MAP@100 : 0.04227 
         nRR@100 : 0.20590 
        nDCG@100 : 0.15845
